Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import ast # used to convert strings to lists

First, read in the imdb data and the giantbomb data as dataframes and get a count

In [2]:
imdb_games = pd.read_csv("imdb_games_db.csv", index_col = 0)
giantbomb_games = pd.read_csv("clean_giantbomb_games_db.csv", index_col = 0)

In [3]:
print("Total imdb_games: {} \nTotal giantbomb_games: {}".format(imdb_games.shape[0], giantbomb_games.shape[0]))

Total imdb_games: 14435 
Total giantbomb_games: 35117


We'll remove any exact duplicates from both dataframes

In [4]:
imdb_games = imdb_games.drop_duplicates()
giantbomb_games = giantbomb_games.drop_duplicates()
print("Total imdb_games: {} \nTotal giantbomb_games: {}".format(imdb_games.shape[0], giantbomb_games.shape[0]))

Total imdb_games: 14435 
Total giantbomb_games: 35116


Have a look at what our dataframes contain:

In [5]:
imdb_games.sample()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas
303923,tt0317518,videoGame,Football,Football,0,1978,\N,\N,"Family,Sport",3.9,24.0,['Football']


In [6]:
giantbomb_games.sample()

,franchise_id,game_id,name,release_year,aliases,developers,genres,platforms,publishers,rating
6502,3025-173,458,X-Men: Wolverine's Rage,NaN,NaN,"['Digital Eclipse Software, Inc.']","['Action', 'Platformer']",['Game Boy Color'],['Activision'],['ESRB: E']


### Remove imdb duplicates

We want to remove any duplicates from our imdb data that contain the same name and year. We save the one with the highest number of votes

In [7]:
imdb_games = imdb_games.sort_values(by=["primaryTitle", "startYear", "numVotes"])
imdb_games = imdb_games.drop_duplicates(subset=["primaryTitle", "startYear"], keep="last")
imdb_games.tconst.count()

14384

In [8]:
imdb_games.primaryTitle[imdb_games.duplicated(["primaryTitle", "startYear"], keep=False)].count()

0

### Remove giantbomb duplicates

As each game_id is unique, we want to group all the franchises for each game into a single result for each game_id. First though we check whether any of the the rows have the same game_id but a different name or release_year

In [9]:
giantbomb_games.count()

franchise_id    35116
game_id         35116
name            35116
release_year     5435
aliases          8088
developers      32746
genres          33660
platforms       34802
publishers      33604
rating          14184
dtype: int64

In [10]:
print(giantbomb_games.groupby("game_id").filter(lambda x: x["name"].nunique() > 1 or x["release_year"].nunique() > 1)["game_id"].count())

0


The next thing will be to remove any franchises which only contain 1 or 2 entries. Though they may technically count as a franchise, they do nothing to help us with analysis on the basis of longevity so we'll treat them as if they don't belong in a franchise

In [11]:
giantbomb_games.game_id.count()

35116

In [12]:
franchise_counts = giantbomb_games.groupby('franchise_id').size()
franchise_dict = {k:v for k,v in franchise_counts.items() if v < 3}
franchise_ids_to_drop = list(franchise_dict.keys())

mask = giantbomb_games['franchise_id'].isin(franchise_ids_to_drop)
giantbomb_games = giantbomb_games.drop(index=giantbomb_games[mask].index)

In [13]:
giantbomb_games.count()

franchise_id    31529
game_id         31529
name            31529
release_year     4668
aliases          7423
developers      29402
genres          30205
platforms       31251
publishers      30282
rating          13149
dtype: int64

The first thing to do is to combine all the games that have an identical franchise_id, release_year and name. With this criteria we will assume they are all the same game released on different platforms for example. We want to consider the release_year even when it is a nan value, so we convert all nan values to 0 accommodate this. Once we've done this, we group by game_id to combine the franchises for each individual game

In [14]:
giantbomb_games = giantbomb_games.groupby(["name", "franchise_id", giantbomb_games["release_year"].fillna(0)]).agg({
    "aliases": lambda x: list(set(x)),
    "developers": lambda x: list(set(x)),
    "genres": lambda x: list(set(x)),
    "platforms": lambda x: list(set(x)),
    "publishers": lambda x: list(set(x)),
    "rating": lambda x: list(set(x)),
    "game_id": "first"
}).reset_index().drop_duplicates(["game_id", "name", "franchise_id", "release_year"], keep="first")

giantbomb_games = giantbomb_games.groupby('game_id').agg({
    'franchise_id': lambda x: list(x),
    'name': 'first',
    'release_year': 'first',
    'aliases': 'first',
    'developers': 'first',
    'genres': 'first',
    "platforms": "first",
    "publishers" : "first",
    "rating": "first"
})

# Change year to string to match imdb database
giantbomb_games["release_year"] = giantbomb_games["release_year"].apply(lambda x : str(x).split('.')[0])

In [15]:
giantbomb_games.sort_values("game_id")

,franchise_id,name,release_year,aliases,developers,genres,platforms,publishers,rating
game_id,,,,,,,,,
1,"[3025-143, 3025-2573]",Desert Strike: Return to the Gulf,1992,[Desert Strike Advance],"[['Electronic Arts', 'Visual Concepts', 'Budca...","[['Action', ""Shoot 'Em Up""]]","[['Amiga', 'Game Boy', 'Game Boy Advance', 'Ga...","[['Electronic Arts', 'Domark Software', 'Malib...","[['ESRB: K-A', 'ESRB: E']]"
3,[3025-2060],Hyperballoid Deluxe: Survival Pack,0,[nan],[['Kernel Kaput']],"[['Action', 'Block-Breaking']]",[['PC']],"[['Alawar Entertainment, Inc.']]",[nan]
4,[3025-1110],The Chessmaster 2000,1986,[nan],"[['The Software Toolworks', 'Software Country'...","[['Strategy', 'Trivia/Board Game']]","[['Amiga', 'Amstrad CPC', 'Apple II', 'Atari S...","[['The Software Toolworks', 'Software Country'...",[nan]
6,"[3025-128, 3025-130, 3025-1372]",WWE SmackDown! vs. RAW 2007,0,[SVR 2007],"[[""Yuke's Co. Ltd."", 'Digital Hearts Co., Ltd....","[['Action', 'Sports', 'Wrestling']]","[['PlayStation Portable', 'PlayStation 2', 'Xb...",[['THQ']],"[['ESRB: T', 'PEGI: 16+', 'CERO: C']]"
8,[3025-335],Super Spy Hunter,0,[Battle Formula],[['Tokai Engineering']],[['Vehicular Combat']],[['Nintendo Entertainment System']],[['Sunsoft']],[nan]
...,...,...,...,...,...,...,...,...,...
88822,[3025-2291],Winning Post 10,0,[nan],[nan],[['Driving/Racing']],"[['PC', 'PlayStation 4', 'Nintendo Switch', 'P...",[['Koei Tecmo']],[['CERO: A']]
88824,[3025-5696],Wan Nyan Dōbutsu Byōin,0,[nan],[nan],[['Simulation']],[['Game Boy Advance']],[['TDK Core']],[['CERO: All Ages']]
88831,[3025-5699],Shogi Saikyou: Pro ni Manabu,0,[nan],[['Magical Company']],[['Trivia/Board Game']],[['PlayStation']],[['Magical Company']],[nan]


We can now get a record of the number of duplicate names in the data

In [16]:
giantbomb_games.name[giantbomb_games.duplicated(["name"], keep=False)].count()

560

We can add the criteria to distinguish by both name and release year, which returns the following:

In [17]:
giantbomb_games.name[giantbomb_games.duplicated(["name", "release_year"], keep=False)].count()

72

We'll combine any duplicates in giantbomb which have the same name and release year

In [18]:
# group by name and release year, aggregate the franchise_id lists
giantbomb_games = giantbomb_games.groupby(['name', 'release_year'], as_index=False).agg({
    'franchise_id': lambda x: list(set([i for l in x for i in l])),
    'name': 'first',
    'release_year': 'first',
    "aliases": 'first',
    "developers": lambda x: list(set([i for l in x for i in l])),
    "genres": lambda x: list(set([i for l in x for i in l])),
    "platforms": lambda x: list(set([i for l in x for i in l])),
    "publishers" : lambda x: list(set([i for l in x for i in l])),
    "rating": lambda x: list(set([i for l in x for i in l]))
})

The release year needs to be converted to a string so we can compare it to the imdb database. We also need to tmove trailing decimal points.

In [19]:
giantbomb_games["release_year"] = giantbomb_games["release_year"].apply(lambda x : str(x).split('.')[0])

Reset the index so we have access to the game_ids

In [20]:
giantbomb_games = giantbomb_games.reset_index()
giantbomb_games = giantbomb_games.rename(columns={'index': 'game_id'})

In [21]:
giantbomb_games.sample()

,game_id,franchise_id,name,release_year,aliases,developers,genres,platforms,publishers,rating
14967,14967,[3025-3081],Ordyne,0,[nan],[['Namco']],"[[""Shoot 'Em Up""]]","[['PlayStation', 'TurboGrafx-16', 'Arcade', 'W...","[['Bandai Namco Entertainment', 'Namco', 'NEC ...","[['ESRB: E', 'CERO: A']]"


## Beginning to combine dataframes

Merge any that have the same title and year

In [22]:
merged_games = pd.merge(imdb_games, giantbomb_games, left_on=['primaryTitle', 'startYear'], right_on=['name', 'release_year'], how='left')
merged_games.game_id.count()

664

In [23]:
additional_merged = pd.merge(imdb_games, giantbomb_games[~giantbomb_games.game_id.isin(merged_games.game_id)], left_on=['originalTitle', 'startYear'], right_on=['name', 'release_year'], how='inner')
merged_games = pd.concat([merged_games, additional_merged], axis=0)
merged_games.game_id.count()

683

In [24]:
def imdb_all_names(imdb_row):
    imdb_row_list = [imdb_row.primaryTitle, imdb_row.originalTitle]
    if type(imdb_row.akas) ==str:
        akas_list = ast.literal_eval(imdb_row.akas)
        for x in akas_list:
            imdb_row_list.append(x)
    return imdb_row_list

Create a row with all the possible names

In [25]:
imdb_games['all_names'] = imdb_games.apply(lambda row: imdb_all_names(row), axis=1)


Remove duplicates from all names and explode

In [26]:
imdb_games['all_names'] = imdb_games['all_names'].apply(lambda x: list(set(x)))

In [27]:
imdb_games = imdb_games.explode('all_names')

Do the same for giantbomb

In [28]:
def gb_all_names(gb_row):
    gb_row_list = [gb_row["name"]]
    for alias in gb_row.aliases:
        if type(alias) == str:
            gb_row_list.append(alias) 
    return gb_row_list

In [29]:
giantbomb_games['all_names'] = giantbomb_games.apply(lambda row: gb_all_names(row), axis=1)
giantbomb_games['all_names'] = giantbomb_games['all_names'].apply(lambda x: list(set(x)))

In [30]:
giantbomb_games = giantbomb_games.explode("all_names")
giantbomb_games_list = giantbomb_games.all_names.to_list()

In [31]:
def get_top_matches(imdb_row):
    return process.extract(imdb_row.all_names, giantbomb_games_list, limit = 3)

In [32]:
import time

In [34]:
from rapidfuzz import process, fuzz

In [35]:
time_at_start = time.perf_counter()
imdb_games['best_matches'] = imdb_games.apply(lambda row: get_top_matches(row), axis = 1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

4590.877031399999


In [37]:
def best_fit(row):
    highest = 0
    best_match = ()
    for match in row["best_matches"]:
        if match[1] > highest:
            highest = match[1]
            best_match = match
    return best_match
imdb_games['best_fit'] = imdb_games.apply(lambda row: best_fit(row), axis = 1)

In [38]:
imdb_games[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = imdb_games['best_fit'].apply(lambda x: pd.Series([i for i in x]))
imdb_games

<ipython-input-38-f913b9cef470>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  imdb_games[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = imdb_games['best_fit'].apply(lambda x: pd.Series([i for i in x]))


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,best_matches,best_fit,best_fit_title,best_fit_ratio,best_fit_game_id
7212109,tt4354918,videoGame,#IDARB (It Draws a Red Box),#IDARB (It Draws a Red Box),0,2015,\N,\N,Action,5.8,25.0,['#IDARB (It Draws a Red Box)'],#IDARB (It Draws a Red Box),[(12 Labours of Hercules IX: A Hero's Moonwalk...,"(12 Labours of Hercules IX: A Hero's Moonwalk,...",12 Labours of Hercules IX: A Hero's Moonwalk,85.5,94.0
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Hyper Sports Special,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28.0
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Track & Field '88,"[(Track & Field, 95.0, 28467), ('88 Games, 85....","(Track & Field, 95.0, 28467)",Track & Field,95.0,28467.0
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",'88 Games,"[('88 Games, 100.0, 27), (ME, 90.0, 15253), ( ...","('88 Games, 100.0, 27)",'88 Games,100.0,27.0
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Konami '88,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295707,tt0308989,videoGame,ssn,ssn,0,1996,\N,\N,\N,7.9,13.0,['ssn'],ssn,"[(N, 90.0, 16876), (N+, 90.0, 16880), (N++, 90...","(N, 90.0, 16876)",N,90.0,16876.0
770432,tt0795512,videoGame,Æon Flux,Æon Flux,0,2005,\N,\N,"Action,Adventure,Sci-Fi",5.6,265.0,['Æon Flux'],Æon Flux,"[(Bit.Trip Flux, 85.5, 2747), (Pool Nation FX,...","(Bit.Trip Flux, 85.5, 2747)",Bit.Trip Flux,85.5,2747.0
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Ihatovo Story,"[(ST, 90.0, 22747), (STO, 90.0, 24242), (TOR, ...","(ST, 90.0, 22747)",ST,90.0,22747.0
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Îhatôvo monogatari,[(Shin Rorona no Atelier: Hajimari no Monogata...,(Shin Rorona no Atelier: Hajimari no Monogatar...,Shin Rorona no Atelier: Hajimari no Monogatari...,85.5,1764.0


In [39]:
imdb_games.to_csv("imdb_games_with_matches.csv")


In [40]:
imdb_games.sort_values("best_fit_ratio")

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,best_matches,best_fit,best_fit_title,best_fit_ratio,best_fit_game_id
1280161,tt10562854,videoGame,Elden Ring,Elden Ring,0,2022,\N,\N,"Action,Adventure,Fantasy",9.4,6839.0,"['Elden Ring', '艾尔登法环']",艾尔登法环,[(Thirty Days & Seven Seas – Pirate Battle Boa...,(Thirty Days & Seven Seas – Pirate Battle Boar...,Thirty Days & Seven Seas – Pirate Battle Board...,12.000000,4317.0
1990972,tt11839496,videoGame,Record of Lodoss War: Deedlit in Wonder Labyrinth,Rôdosutou senki: Dîdoritto in wandâ rabirinsu,0,2020,\N,\N,"Action,Adventure,Fantasy",7.4,17.0,['Record of Lodoss War: Deedlit in Wonder Laby...,蒂德莉特的奇境冒险,[(Guardian's Sword Plus\nGuardian Sword EX\n守護...,(Guardian's Sword Plus\nGuardian Sword EX\n守護者...,Guardian's Sword Plus\nGuardian Sword EX\n守護者之...,15.000000,10515.0
399734,tt0416843,videoGame,Illbleed,Illbleed,0,2001,\N,\N,"Action,Adventure,Horror",8.3,71.0,"['Illbleed', 'イルブリード', '勇闖鬼界']",勇闖鬼界,[(Thor: TDW - The Official Game\r\nThor 2: The...,(Thor: TDW - The Official Game\r\nThor 2: The ...,Thor: TDW - The Official Game\r\nThor 2: The D...,15.000000,27764.0
272425,tt0284556,videoGame,Theme Park,Theme Park,0,1994,\N,\N,"Action,Comedy,Family",7.5,269.0,"['Theme Park', 'テーマパーク', '主题公园']",主题公园,[(Captain America: TWS\r\n캡틴 아메리카: 윈터 솔져 - 영화공...,(Captain America: TWS\r\n캡틴 아메리카: 윈터 솔져 - 영화공식...,Captain America: TWS\r\n캡틴 아메리카: 윈터 솔져 - 영화공식게...,15.000000,3740.0
7991269,tt6116988,videoGame,Super Mario World,Super Mario World,0,1995,\N,\N,Action,7.5,19.0,"['45-in-1 version', '超級馬里奧世界', 'Super Mario Wo...",超級馬里奧世界,[(Thor: TDW - The Official Game\r\nThor 2: The...,(Thor: TDW - The Official Game\r\nThor 2: The ...,Thor: TDW - The Official Game\r\nThor 2: The D...,17.142857,27764.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7854274,tt5807606,videoGame,Death Stranding,Death Stranding,0,2019,\N,\N,"Action,Adventure,Drama",8.8,9494.0,"['Death Stranding', '死亡擱淺']",死亡擱淺,[( 2 Games In 1: LEGO Knights' Kingdom + LEGO ...,(),NaN,NaN,NaN
9691482,tt9843598,videoGame,Detention,Huan yuan,0,2019,\N,\N,"Drama,Horror,Mystery",8.1,94.0,"['Detention', 'Devotion', 'Huan yuan', '還願']",還願,[( 2 Games In 1: LEGO Knights' Kingdom + LEGO ...,(),NaN,NaN,NaN
9135966,tt8638132,videoGame,Hello Neighbor,Hello Neighbor,0,2017,\N,\N,"Family,Fantasy,Horror",5.3,423.0,"['Hello Neighbor', '你好邻居']",你好邻居,[( 2 Games In 1: LEGO Knights' Kingdom + LEGO ...,(),NaN,NaN,NaN
282276,tt0294861,videoGame,Q*bert,Q*bert,0,1982,\N,\N,Family,6.6,181.0,"['@!#?@!', 'Q*bert', 'Cubes', 'Snots and Booge...",@!#?@!,[( 2 Games In 1: LEGO Knights' Kingdom + LEGO ...,(),NaN,NaN,NaN


Drop any nan values for best_fit_ratio as these are problem entries where the titles are messed up

In [44]:
imdb_games = imdb_games[imdb_games['best_fit_ratio'].notna()]

Convert the game ids to ints

In [47]:
imdb_games["best_fit_game_id"] = imdb_games["best_fit_game_id"].apply(lambda x: int(x))

<ipython-input-47-f5b85e1c8027>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_games["best_fit_game_id"] = imdb_games["best_fit_game_id"].apply(lambda x: int(x))


In [48]:
imdb_games

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,best_matches,best_fit,best_fit_title,best_fit_ratio,best_fit_game_id
7212109,tt4354918,videoGame,#IDARB (It Draws a Red Box),#IDARB (It Draws a Red Box),0,2015,\N,\N,Action,5.8,25.0,['#IDARB (It Draws a Red Box)'],#IDARB (It Draws a Red Box),[(12 Labours of Hercules IX: A Hero's Moonwalk...,"(12 Labours of Hercules IX: A Hero's Moonwalk,...",12 Labours of Hercules IX: A Hero's Moonwalk,85.5,94
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Hyper Sports Special,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Track & Field '88,"[(Track & Field, 95.0, 28467), ('88 Games, 85....","(Track & Field, 95.0, 28467)",Track & Field,95.0,28467
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",'88 Games,"[('88 Games, 100.0, 27), (ME, 90.0, 15253), ( ...","('88 Games, 100.0, 27)",'88 Games,100.0,27
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Konami '88,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295707,tt0308989,videoGame,ssn,ssn,0,1996,\N,\N,\N,7.9,13.0,['ssn'],ssn,"[(N, 90.0, 16876), (N+, 90.0, 16880), (N++, 90...","(N, 90.0, 16876)",N,90.0,16876
770432,tt0795512,videoGame,Æon Flux,Æon Flux,0,2005,\N,\N,"Action,Adventure,Sci-Fi",5.6,265.0,['Æon Flux'],Æon Flux,"[(Bit.Trip Flux, 85.5, 2747), (Pool Nation FX,...","(Bit.Trip Flux, 85.5, 2747)",Bit.Trip Flux,85.5,2747
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Ihatovo Story,"[(ST, 90.0, 22747), (STO, 90.0, 24242), (TOR, ...","(ST, 90.0, 22747)",ST,90.0,22747
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Îhatôvo monogatari,[(Shin Rorona no Atelier: Hajimari no Monogata...,(Shin Rorona no Atelier: Hajimari no Monogatar...,Shin Rorona no Atelier: Hajimari no Monogatari...,85.5,1764


In [60]:
giantbomb_games.head(50)

,game_id,franchise_id,name,release_year,aliases,developers,genres,platforms,publishers,rating,all_names
0,0,"[3025-725, 3025-448]",2 Games In 1: LEGO Knights' Kingdom + LEGO Bi...,0,[nan],"[['Razorback Developments Ltd', 'Rockstar Leed...",[['Action']],[['Game Boy Advance']],[['THQ']],[nan],2 Games In 1: LEGO Knights' Kingdom + LEGO Bi...
1,1,[3025-647],Atelier Ryza 2: Lost Legends & the Secret Fairy,0,[nan],"[['GUST Co., Ltd.']]",[['Role-Playing']],"[['PC', 'PlayStation 4', 'Nintendo Switch', 'P...",[['Koei Tecmo']],"[['CERO: C', 'ESRB: T']]",Atelier Ryza 2: Lost Legends & the Secret Fairy
2,2,"[3025-1691, 3025-1819]",Electric Crayon 3.0: Inspector Gadget: Safety...,0,[nan],"[['Brian A. Rice, Inc.']]",[['Educational']],[['PC']],[['Merit Software']],[nan],Electric Crayon 3.0: Inspector Gadget: Safety...
3,3,[3025-3748],Empire Deluxe Combined Edition,0,[Empire Deluxe Masters Edition\nEmpire Deluxe ...,"[['White Wolf Productions', 'Killer Bee Softwa...",[['Strategy']],[['PC']],"[['New World Computing, Inc.', 'Killer Bee Sof...",[nan],Empire Deluxe Masters Edition\nEmpire Deluxe I...
3,3,[3025-3748],Empire Deluxe Combined Edition,0,[Empire Deluxe Masters Edition\nEmpire Deluxe ...,"[['White Wolf Productions', 'Killer Bee Softwa...",[['Strategy']],[['PC']],"[['New World Computing, Inc.', 'Killer Bee Sof...",[nan],Empire Deluxe Combined Edition
4,4,[3025-805],F1 2019,0,[nan],[['Codemasters']],[['Driving/Racing']],"[['PC', 'Xbox One', 'PlayStation 4']]","[['Codemasters', 'Game Source Entertainment']]",[['ESRB: E']],F1 2019
5,5,[3025-3067],Fun School: Maths,1997,[nan],[['Intellectual Software Consultants Limited']],[['Educational']],[['PC']],"[[""McVitie's Penguin"", 'Q']]",[nan],Fun School: Maths
6,6,[3025-5624],Gokujou!! Mecha Mote Iinchou: MM My Best Friend!,0,[nan],[['HuneX']],[['Simulation']],[['Nintendo DS']],[['Konami']],[['CERO: A']],Gokujou!! Mecha Mote Iinchou: MM My Best Friend!
7,7,[3025-5510],Illvelo Dillinjah,0,[nan],[['RS34']],"[[""Shoot 'Em Up""]]",[['Nintendo 3DS eShop']],[['Dual']],[nan],Illvelo Dillinjah
8,8,[3025-4877],Imagicard: PAW Patrol,0,[nan],[nan],[['Educational']],[['LeapPad']],[['LeapFrog Enterprises']],[nan],Imagicard: PAW Patrol


Remove the extra lists from the developers, genres, platforms, publishers and rating columns

In [63]:
giantbomb_games["developers"] = giantbomb_games["developers"].apply(lambda x: x[0])
giantbomb_games["genres"] = giantbomb_games["genres"].apply(lambda x: x[0])
giantbomb_games["platforms"] = giantbomb_games["platforms"].apply(lambda x: x[0])
giantbomb_games["publishers"] = giantbomb_games["publishers"].apply(lambda x: x[0])
giantbomb_games["rating"] = giantbomb_games["rating"].apply(lambda x: x[0])

# Protecting myself - using copied of DFs

In [268]:
copy_of_imdb_games = imdb_games

In [269]:
copy_of_giantbomb_games = giantbomb_games

Drop the index, so we can use the index from the exploded giantbomb dataframe (which will directly line up with the position in the list where the matches are made)

In [270]:
copy_of_giantbomb_games = copy_of_giantbomb_games.reset_index(drop = True)

Merge all the titles, taking the best fit for each game and applying to the imdb dataset

In [271]:
copy_of_imdb_games = pd.merge(copy_of_imdb_games, copy_of_giantbomb_games,  left_on=["best_fit_game_id"], right_index=True, how="left")
copy_of_imdb_games

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,averageRating,numVotes,akas,all_names_x,best_matches,best_fit,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
7212109,tt4354918,videoGame,#IDARB (It Draws a Red Box),#IDARB (It Draws a Red Box),0,2015,\N,\N,Action,5.8,25.0,['#IDARB (It Draws a Red Box)'],#IDARB (It Draws a Red Box),[(12 Labours of Hercules IX: A Hero's Moonwalk...,"(12 Labours of Hercules IX: A Hero's Moonwalk,...",12 Labours of Hercules IX: A Hero's Moonwalk,85.5,94,79,[3025-4692],12 Labours of Hercules IX: A Hero's Moonwalk,0,[nan],"['Jetdogs Studios', 'Zoom Out Games']","['Strategy', 'Puzzle']",['PC'],['Jetdogs Studios'],NaN,12 Labours of Hercules IX: A Hero's Moonwalk
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Hyper Sports Special,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28,26,[3025-1053],'88 Games,1988,[Konami '88\r\nHyper Sports Special],['Konami'],['Track & Field'],['Arcade'],['Konami'],NaN,Konami '88\r\nHyper Sports Special
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Track & Field '88,"[(Track & Field, 95.0, 28467), ('88 Games, 85....","(Track & Field, 95.0, 28467)",Track & Field,95.0,28467,23252,"[3025-3709, 3025-1053]",Track & Field,1983,[Hyper Olympic\r\nTrack and Field\r\nKonamic S...,"['Konami', 'Atari', 'K-Byte']","['Sports', 'Track & Field']","['Game Boy', 'Apple II', 'Commodore 64', 'Nint...","['Konami', 'Atari', 'Centuri, Inc.', 'Gradiente']",['ESRB: E'],Track & Field
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",'88 Games,"[('88 Games, 100.0, 27), (ME, 90.0, 15253), ( ...","('88 Games, 100.0, 27)",'88 Games,100.0,27,26,[3025-1053],'88 Games,1988,[Konami '88\r\nHyper Sports Special],['Konami'],['Track & Field'],['Arcade'],['Konami'],NaN,'88 Games
5075818,tt1968978,videoGame,'88 Games,Hyper Sports Special,0,1988,\N,\N,Sport,6.0,14.0,"[""Track & Field '88"", 'Hyper Sports Special', ...",Konami '88,"[(Konami '88\r\nHyper Sports Special, 90.0, 28...","(Konami '88\r\nHyper Sports Special, 90.0, 28)",Konami '88\r\nHyper Sports Special,90.0,28,26,[3025-1053],'88 Games,1988,[Konami '88\r\nHyper Sports Special],['Konami'],['Track & Field'],['Arcade'],['Konami'],NaN,Konami '88\r\nHyper Sports Special
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295707,tt0308989,videoGame,ssn,ssn,0,1996,\N,\N,\N,7.9,13.0,['ssn'],ssn,"[(N, 90.0, 16876), (N+, 90.0, 16880), (N++, 90...","(N, 90.0, 16876)",N,90.0,16876,13855,[3025-870],N,0,[N-Game\r\nNNNNNNNNNNNNN],['Metanet Software'],['Platformer'],"['PC', 'Browser']",['Metanet Software'],NaN,N
770432,tt0795512,videoGame,Æon Flux,Æon Flux,0,2005,\N,\N,"Action,Adventure,Sci-Fi",5.6,265.0,['Æon Flux'],Æon Flux,"[(Bit.Trip Flux, 85.5, 2747), (Pool Nation FX,...","(Bit.Trip Flux, 85.5, 2747)",Bit.Trip Flux,85.5,2747,2292,[3025-1298],Bit.Trip Flux,0,[nan],['Choice Provisions'],['Music/Rhythm'],"['Wii Shop', 'PC']",['Aksys Games'],NaN,Bit.Trip Flux
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Ihatovo Story,"[(ST, 90.0, 22747), (STO, 90.0, 24242), (TOR, ...","(ST, 90.0, 22747)",ST,90.0,22747,18630,[3025-3153],Shining Tears,0,[ST],['Nex Entertainment'],"['Action', 'Role-Playing']",['PlayStation 2'],['Sega'],['ESRB: T'],ST
2185219,tt1219283,videoGame,Îhatôvo monogatari,Îhatôvo monogatari,0,1993,\N,\N,Adventure,7.0,6.0,"['Ihatovo Story', 'Îhatôvo monogatari']",Îhatôvo monogatari,[(Shin R

Remove unneccessary columns

In [272]:
copy_of_imdb_games = copy_of_imdb_games.drop(columns = ["endYear", "best_matches", "best_fit", "runtimeMinutes", "all_names_y", "aliases", "akas"])

Sort the values by tconst and best_fit_ratio. We then want to keep the highest best_fit for each tconst game

In [273]:
copy_of_imdb_games = copy_of_imdb_games.sort_values(["tconst", "best_fit_ratio"])
copy_of_imdb_games = copy_of_imdb_games.drop_duplicates(["tconst"], keep="last")

In [274]:
copy_of_imdb_games

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,developers,genres_y,platforms,publishers,rating
82552,tt0084376,videoGame,"MysteryDisc: Murder, Anyone?","MysteryDisc: Murder, Anyone?",0,1982,"Adventure,Crime,Mystery",6.1,37.0,"MysteryDisc: Murder, Anyone?",Myst,90.0,16792,13790,"[3025-527, 3025-3987]",Myst,0,['Cyan'],"['Adventure', 'Puzzle']","['Amiga', 'Mac', 'PlayStation Portable', 'Play...","['Brøderbund Software, Inc.']","['ESRB: K-A', 'PEGI: 3+', 'CERO: All Ages', 'E..."
102667,tt0105000,videoGame,Night Trap,Night Trap,0,1992,"Adventure,Horror,Mystery",6.2,376.0,Night Trap,A Better Mouse Trap,85.5,308,273,[3025-3963],A Better Mouse Trap,0,['Dave Gilbert'],['Adventure'],['PC'],['Dave Gilbert'],NaN
107391,tt0109865,videoGame,Gabriel Knight: Sins of the Fathers,Gabriel Knight: Sins of the Fathers,0,1993,"Adventure,Drama,Horror",9.1,706.0,Gabriel Knight: Sins of the Fathers,Gabriel Knight: Sins of the Fathers,100.0,9591,7821,[3025-638],Gabriel Knight: Sins of the Fathers,0,"['Sierra', 'Phoenix Online Studios', 'Pinkerto...",['Adventure'],"['Mac', 'PC', 'iPhone', 'iPad', 'Android']","['Sierra', 'Pinkerton Road Studio']",['ESRB: T']
107783,tt0110267,videoGame,King's Quest VII: The Princeless Bride,King's Quest VII: The Princeless Bride,0,1994,"Adventure,Fantasy",7.6,206.0,King's Quest VII: The Princeless Bride,King's Quest VII: The Princeless Bride,100.0,13304,10917,[3025-413],King's Quest VII: The Princeless Bride,0,['Sierra'],['Adventure'],"['Mac', 'PC']",['Sierra'],"['ESRB: E', 'ESRB: K-A', 'ESRB: T']"
108416,tt0110909,videoGame,Psychic Detective,Psychic Detective,0,1995,"Adventure,Fantasy,Mystery",8.5,50.0,Psychic Detective,Psychic Detective Final: Solitude Part 2,90.0,20042,16427,[3025-1867],Psychic Detective Final: Solitude Part 2,0,['DataWest'],['Adventure'],['FM Towns'],['DataWest'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9712195,tt9888864,videoGame,Infernal,Infernal,0,2007,Action,6.0,17.0,Infernal,Indiana Jones and the Infernal Machine,90.0,12020,9806,[3025-358],Indiana Jones and the Infernal Machine,0,"['HotGen Ltd.', 'LucasArts Entertainment Compa...",['Action-Adventure'],"['Nintendo 64', 'Game Boy Color', 'PC']","['THQ', 'LucasArts Entertainment Company LLC']","['ESRB: T', 'ESRB: E']"
9713877,tt9892552,videoGame,Spiral Splatter,Spiral Splatter,0,2017,Action,2.0,10.0,Spiral Splatter,Majin Tensei 2: Spiral Nemesis,85.5,14910,12274,"[3025-1123, 3025-525]",Majin Tensei II: Spiral Nemesis,0,['Atlus'],"['Strategy', 'Role-Playing']",['Super Nintendo Entertainment System'],['Atlus'],['CERO: A']
9714234,tt9893348,videoGame,Alien: Covenant In Utero VR Experience,Alien: Covenant In Utero VR Experience,0,2017,Sci-Fi,6.2,17.0,Alien: Covenant In Utero VR Experience,Alien,90.0,874,732,[3025-25],Alien,0,['Argonaut Games'],['Strategy'],"['Amstrad CPC', 'Commodore 64', 'ZX Spectrum',...",['Argus Press Software Ltd.'],NaN
9714437,tt9893804,videoGame,The Lego Movie 2 Videogame,The Lego Movie 2 Videogame,0,2019,"Adventure,Animation,Family",5.5,156.0,The Lego Movie 2 Videogame,The LEGO Movie 2 Videogame,100.0,26962,22035,[3025-448],The LEGO Movie 2 Videogame,0,['TT Games'],['Action-Adventure'],"['PC', 'Xbox One', 'PlayStation 4', 'Nintendo ...",['WB Games'],NaN


Now I need to remove any records where the best_fit_ratio is too low. I can examine some of the criteria. When looking at these, we compare all_names_x and best_fit_title as these are the 2 titles that the ratio is calculated on.

In [240]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 75].sort_values("best_fit_ratio").head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
1911092,tt11696500,videoGame,Boneworks,Boneworks,0,2019,"Action,Adventure,Sci-Fi",8.1,126.0,['Boneworks'],Boneworks,Bonkers,75.0,3135,2613,"[3025-3303, 3025-720]",Bonkers,0,[nan],['SEGA Interactive Development Division'],"['Action', 'Driving/Racing']",['Genesis'],['Sega'],NaN,Bonkers
439140,tt0457320,videoGame,Deadhunt,Deadhunt,0,2005,"Action,Horror",7.2,46.0,['Deadhunt'],Deadhunt,DreadOut,75.0,7236,5932,[3025-3903],DreadOut,0,[nan],['Digital Happiness'],['Adventure'],"['Mac', 'PC', 'Linux']",['Digital Happiness'],['ESRB: M'],DreadOut
3478047,tt14582896,videoGame,Deadbolt,Deadbolt,0,2016,"Action,Crime,Horror",8.5,8.0,['Deadbolt'],Deadbolt,Deadpool,75.0,5635,4633,[3025-173],Deadpool,0,[nan],"['High Moon Studios, Inc.']",['Action'],"['Xbox 360', 'PlayStation 3', 'Xbox 360 Games ...",['Activision'],['ESRB: M'],Deadpool
5979410,tt23641238,videoGame,Foreclosed,Foreclosed,0,2021,"Action,Adventure",4.6,6.0,NaN,Foreclosed,Forced,75.0,9182,7460,[3025-3226],Forced,0,[nan],['betaDwarf Entertainment'],"['Action', 'Role-Playing', 'Brawler']","['Mac', 'PC', 'Wii U', 'Xbox One', 'PlayStatio...",['betaDwarf Entertainment'],NaN,Forced
2354052,tt12504034,videoGame,Suzerain,Suzerain,0,2020,"Adventure,Drama,History",8.0,8.0,['Suzerain'],Suzerain,Superman,75.0,25505,20836,"[3025-138, 3025-317]",Superman,0,"[nan, Superman 64, Super-Man]",['Atari'],['Action'],['Nintendo 64'],"['Sears, Roebuck and Co.', 'Atari']",NaN,Superman
1280421,tt10563334,videoGame,Misao,Misao,0,2011,"Adventure,Fantasy,Horror",7.2,26.0,['Misao'],Misao,MSA,75.0,15795,12963,[3025-531],Metal Slug Anthology,0,[MSA],"['SNK', 'Nazca Corporation', 'Terminal Reality...","['Action', 'Shooter', 'Compilation']","['PlayStation Portable', 'PlayStation 2', 'Wii...",['SNK'],"['ESRB: T', 'CERO: B', 'OFLC: M', 'PEGI: 12+']",MSA
3325510,tt14304318,videoGame,ALTF4,ALTF4,0,2021,"Action,Animation",7.1,12.0,NaN,ALTF4,ALF,75.0,415,367,[3025-2096],ALF,0,[nan],"['Nexa Corporation', 'Sphere, Inc.']","['Action', 'Adventure']",['Sega Master System'],['Sega'],NaN,ALF
3424965,tt14485778,videoGame,Unturned,Unturned,0,2017,"Action,Adventure,Fantasy",5.9,56.0,['Unturned'],Unturned,Nocturne,75.0,17972,14762,[3025-3299],Nocturne,1986,[nan],['Alpha-Omega Software'],['Text Adventure'],['Amstrad CPC'],['Alpha-Omega Software'],NaN,Nocturne
2961693,tt13639614,videoGame,Helltaker,Helltaker,0,2020,Action,6.9,111.0,NaN,Helltaker,Shelter,75.0,22555,18489,[3025-2979],Shelter,0,[nan],['Might and Delight'],['Adventure'],"['Mac', 'PC']",['Might and Delight'],['ESRB: E10+'],Shelter
2918540,tt13560088,videoGame,Sneaky Sasquatch,Sneaky Sasquatch,0,2019,\N,8.4,15.0,['Sneaky Sasquatch'],Sneaky Sasquatch,Quarth,75.0,20305,16639,"[3025-3081, 3025-3563]",Quarth,1989,[Block Hole\nArcade Archive: Block Hole\nArcad...,['Konami'],"['Action', 'Puzzle']","['Game Boy', 'MSX', 'PlayStation 2', 'Nintendo...","['Konami', 'HAMSTER Corporation']","['ESRB: E', 'CERO: A']",Quarth


At 75%, not one of our entries seems like a sensible match. We'll try with 85%

In [241]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 85].sort_values("best_fit_ratio").head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
3613808,tt14834068,videoGame,Two Point Campus,Two Point Campus,0,2022,\N,7.5,16.0,['Two Point Campus'],Two Point Campus,Amazing Adventures: Riddle of the Two Knights,85.5,1024,855,[3025-2698],Amazing Adventures: Riddle of the Two Knights,0,[nan],['SpinTop Games'],"['Adventure', 'Puzzle']","['Mac', 'PC']","['PopCap Games, Inc.']",NaN,Amazing Adventures: Riddle of the Two Knights
398662,tt0415743,videoGame,Demons to Diamonds,Demons to Diamonds,0,1978,\N,5.1,11.0,['Demons to Diamonds'],Demons to Diamonds,0-Ji No Kane To Cinderella: Halloween Wedding,85.5,46,41,[3025-3799],0-Ji No Kane To Cinderella: Halloween Wedding,0,[０時の鐘とシンデレラ～Halloween Wedding～],['QuinRose'],['Adventure'],"['PlayStation Portable', 'PlayStation Vita']",['QuinRose'],['CERO: C'],0-Ji No Kane To Cinderella: Halloween Wedding
5218960,tt20317198,videoGame,Michaelsoft Binbows '97,Michaelsoft Binbows '97,0,2022,"Comedy,Horror",7.8,8.0,"[""Michaelsoft Binbows '97""]",Michaelsoft Binbows '97,Adidas Power Soccer International 97,85.5,536,465,[3025-829],Adidas Power Soccer International 97,1997,[nan],['SCEE Studio Liverpool'],"['Sports', 'Soccer']",['PlayStation'],['SCEE Studio Liverpool'],NaN,Adidas Power Soccer International 97
5204402,tt2024369,videoGame,BMW M3 Challenge,BMW M3 Challenge,0,2007,Sport,6.4,9.0,['BMW M3 Challenge'],BMW M3 Challenge,Family Trainer: Extreme Challenge,85.5,503,439,[3025-2406],Active Life: Extreme Challenge,0,[Family Trainer: Extreme Challenge],['Bandai Namco Entertainment'],['Fitness'],['Wii'],['Bandai Namco Entertainment'],['ESRB: E'],Family Trainer: Extreme Challenge
5195512,tt2022483,videoGame,Off Road Challenge,Off Road Challenge,0,1997,"Action,Sport",6.9,14.0,['Off Road Challenge'],Off Road Challenge,Family Trainer: Extreme Challenge,85.5,503,439,[3025-2406],Active Life: Extreme Challenge,0,[Family Trainer: Extreme Challenge],['Bandai Namco Entertainment'],['Fitness'],['Wii'],['Bandai Namco Entertainment'],['ESRB: E'],Family Trainer: Extreme Challenge
5192297,tt20217988,videoGame,Warped Kart Racers,Warped Kart Racers,0,2022,Sport,7.1,10.0,['Warped Kart Racers'],Warped Kart Racers,BC Racers,85.5,1912,1595,[3025-92],BC Racers,1994,[nan],['Core Design Ltd.'],['Driving/Racing'],"['3DO', 'Sega CD', 'Sega 32X', 'PC']",['Core Design Ltd.'],NaN,BC Racers
5182430,tt20191954,videoGame,Oh... Sir!: The Insult Simulator,Oh... Sir!: The Insult Simulator,0,2016,Comedy,5.8,6.0,['Oh... Sir!: The Insult Simulator'],Oh... Sir!: The Insult Simulator,Simple 2000 Series Vol.067: The Suiri - Soshi...,85.5,19,18,"[3025-1172, 3025-4824]",Simple 2000 Series Vol.067: The Suiri - Soshi...,0,[nan],['Tomcat System'],['Adventure'],['PlayStation 2'],['D3 Publisher Inc.'],NaN,Simple 2000 Series Vol.067: The Suiri - Soshi...
5175078,tt2014370,videoGame,The Maw,The Maw,0,2008,Adventure,7.0,26.0,['The Maw'],The Maw,Atelier Ryza 2: Lost Legends & the Secret Fairy,85.5,1,1,[3025-647],Atelier Ryza 2: Lost Legends & the Secret Fairy,0,[nan],"['GUST Co., Ltd.']",['Role-Playing'],"['PC', 'PlayStation 4', 'Nintendo Switch', 'Pl...",['Koei Tecmo'],"['CERO: C', 'ESRB: T']",Atelier Ryza 2: Lost Legends & the Secret Fairy
5173856,tt2013262,videoGame,Purble Place,Purble Place,0,2007,Family,8.3,60.0,"['Peobeul peulleiseu', 'Purble Place']",Purble Place,Physical Contact: Picture Place,85.5,18987,15595,[3025-3970],Physical Contact: Picture Place,0,[nan],NaN,['Puzzle'],"['New Nintendo 3DS', 'Nintendo Switch']",['Collavier Corporation'],NaN,Physical Contact: Picture Place
5161602,tt2006757,videoGame,Heroes Over Europe,Heroes Over Europe,0,2009,"Action,War",6.6,12.0,['Heroes Over Europe'],Heroes Over Europe,Action Force II: International Heroes,85.5,487,425,[3025-223],Action Force II: International Heroes,1988,[nan],['Gang of 

Again, 85% seems to be too obscure. We'll up it to 90%

In [172]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 90].sort_values("best_fit_ratio").head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
82552,tt0084376,videoGame,"MysteryDisc: Murder, Anyone?","MysteryDisc: Murder, Anyone?",0,1982,"Adventure,Crime,Mystery",6.1,37.0,"['MysteryDisc: Murder, Anyone?']","MysteryDisc: Murder, Anyone?",Myst,90.0,16792,13790,"[3025-527, 3025-3987]",Myst,0,[nan],['Cyan'],"['Adventure', 'Puzzle']","['Amiga', 'Mac', 'PlayStation Portable', 'Play...","['Brøderbund Software, Inc.']","['ESRB: K-A', 'PEGI: 3+', 'CERO: All Ages', 'E...",Myst
3552094,tt14719802,videoGame,ESWAT: City Under Siege,Cyber Police ESWAT,0,1990,Action,4.4,10.0,"['ESWAT: City Under Siege', 'サイバーポリス イースワット', ...",ESWAT: City Under Siege,Siege,90.0,22946,18795,[3025-984],Siege,1992,[nan],"['Mindcraft Software, Inc.']","['Strategy', 'Real-Time Strategy']",['PC'],"['Mindcraft Software, Inc.', 'Electronic Arts'...",NaN,Siege
3548688,tt1471341,videoGame,The Godfather II: Crime Rings,The Godfather II: Crime Rings,0,2008,Crime,7.8,42.0,['The Godfather II: Crime Rings'],The Godfather II: Crime Rings,The Godfather II,90.0,26712,21832,[3025-1132],The Godfather II,0,"[The Godfather 2, Le Parrain 2]","['Visceral Games', 'Electronic Arts']","['Strategy', 'Driving/Racing', 'Shooter']","['Xbox 360', 'PlayStation 3', 'PC']","['EA Games', 'Electronic Arts']","['ESRB: M', 'CERO: D', 'PEGI: 18+']",The Godfather II
3548402,tt1471292,videoGame,iD3,iD3,0,2009,"Adventure,Crime,Mystery",7.4,14.0,['iD3'],iD3,D,90.0,5134,4238,[3025-126],D,0,[D's Diner\r\nD's Diner: Director's Cut\r\nD's...,['Warp Inc.'],"['Adventure', 'Puzzle']","['Mac', 'PlayStation', '3DO', 'Saturn', 'PC']","['Acclaim Entertainment, Inc.', 'Nightdive Stu...",['ESRB: T'],D
3548269,tt1471269,videoGame,Solstice II,Solstice II,0,1993,"Adventure,Fantasy",4.7,9.0,"['Solstice II', 'Equinox']",Solstice II,ST,90.0,22747,18630,[3025-3153],Shining Tears,0,[ST],['Nex Entertainment'],"['Action', 'Role-Playing']",['PlayStation 2'],['Sega'],['ESRB: T'],ST
3547949,tt1471213,videoGame,The Battle of Olympus,Oryunposu no tatakai - Ai no densetsu,0,1988,"Adventure,Fantasy",7.7,41.0,"['The Battle of Olympus', 'Oryunposu no tataka...",The Battle of Olympus,Olympus,90.0,27492,22446,[3025-282],The Sims 4,0,[Olympus],"['Maxis Software', 'The Sims Studio']",['Simulation'],"['Mac', 'PC', 'Xbox One', 'PlayStation 4']",['Electronic Arts'],"['ESRB: T', 'OFLC: M', 'PEGI: 12+']",Olympus
3545145,tt1470709,videoGame,Toobin',Toobin',0,1988,"Adventure,Sport",7.3,29.0,"[""Toobin'"", 'Ciambella']",Toobin',N,90.0,16876,13855,[3025-870],N,0,[N-Game\r\nNNNNNNNNNNNNN],['Metanet Software'],['Platformer'],"['PC', 'Browser']",['Metanet Software'],NaN,N
3531725,tt14682236,videoGame,4 Elements II,4 Elements II,0,2011,"Adventure,Family,Fantasy",6.6,5.0,['4 Elements II'],4 Elements II,ME,90.0,15253,12541,[3025-560],Mass Effect,0,[ME],"['BioWare', 'Edge of Reality, Ltd.', 'Demiurge...","['Role-Playing', 'Shooter']","['Xbox 360', 'PlayStation 3', 'PlayStation Net...","['Xbox Game Studios', 'EA Games']","['ESRB: M', 'BBFC: 12', 'OFLC: MA15+', 'CERO: ...",ME
3556634,tt14728214,videoGame,Dangerous Golf,Dangerous Golf,0,2016,Sport,6.2,6.0,['Dangerous Golf'],Dangerous Golf,Golf,90.0,10256,8381,"[3025-2058, 3025-1, 3025-506, 3025-3081]",Golf,0,[nan],['Nintendo R&D1'],"['Sports', 'Golf']","['Game Boy', 'Game Boy Advance', 'Nintendo Ent...","['Nintendo', 'Hudson Entertainment, Inc.', 'HA...","['ESRB: E', 'PEGI: 3+']",Golf
3523888,tt14667638,videoGame,Mafia III: Sign of the Times,Mafia III: Sign of the Times,0,2017,"Action,Crime,Drama",6.5,32.0,['Mafia III: Sign of the Times'],Mafia III: Sign of the Times,Mafia,90.0,14712,12105,[3025-921],Mafia,0,[Mafia: The City of Lost Heaven],"['Illusion Softworks, a.s.']","['Action', 'Driving/Racing', 'Shooter']","['PlayStation 2', 'Xbox', 'PC']",['Gathering of Developers'],"['ESRB: M',

Approximately 6 of those 20 look like they could be a good match for the franchise, even if the game name is not the same. However this equates to a 30% accuracy, which I would not be happy with. 

In [176]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 92].sort_values("best_fit_ratio").head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
3165105,tt1401147,videoGame,SpongeBob's Atlantis SquarePantis,SpongeBob's Atlantis SquarePantis,0,2007,"Action,Adventure,Comedy",6.3,97.0,"['Bob Esponja: Aventura en la Atlántida', 'Ο Μ...",SpongeBob's Atlantis SquarePantis,SpongeBob SquarePants: Atlantis Squarepantis,92.031250,24050,19679,[3025-337],SpongeBob SquarePants: Atlantis Squarepantis,0,[nan],"['THQ', 'Blitz Games Studios']",['Action-Adventure'],"['Game Boy Advance', 'PlayStation 2', 'Wii', '...",['THQ'],"['ESRB: E', 'CERO: A']",SpongeBob SquarePants: Atlantis Squarepantis
5262903,tt2058087,videoGame,Sword of the Stars 2: Lords of Winter,Sword of the Stars 2: Lords of Winter,0,2011,Sci-Fi,4.1,18.0,['Sword of the Stars 2: Lords of Winter'],Sword of the Stars 2: Lords of Winter,Sword of the Stars II: The Lords of Winter,92.031250,25595,20908,[3025-1290],Sword of the Stars II: The Lords of Winter,0,[nan],['Kerberos Productions'],['Strategy'],['PC'],"['Paradox Interactive AB', 'Ikaron']",NaN,Sword of the Stars II: The Lords of Winter
265597,tt0277439,videoGame,Who Wants to Beat Up a Millionaire,Who Wants to Beat Up a Millionaire,0,2000,Comedy,4.4,10.0,['Who Wants to Beat Up a Millionaire'],Who Wants to Beat Up a Millionaire,Who Wants To Be A Millionaire,92.063492,30028,24528,[3025-1956],Who Wants To Be A Millionaire,0,[nan],NaN,NaN,"['PlayStation', 'Game Boy Color', 'PC']",NaN,NaN,Who Wants To Be A Millionaire
259473,tt0270990,videoGame,Hokuto no Ken: Seikimatsu kyûseishu densetsu,Hokuto no Ken: Seikimatsu kyûseishu densetsu,0,2000,\N,7.8,14.0,['Hokuto no Ken: Seikimatsu kyûseishu densetsu'],Hokuto no Ken: Seikimatsu kyûseishu densetsu,Hokuto no Ken: Seiki Matsukyu Seishi Densetsu,92.134831,11445,9318,[3025-1023],Hokuto no Ken: Seiki Matsukyu Seishi Densetsu,0,[nan],['Natsume'],"['Action', 'Brawler']",['PlayStation'],"['Bandai Co., Ltd.']",NaN,Hokuto no Ken: Seiki Matsukyu Seishi Densetsu
8596801,tt7462624,videoGame,Atelier Lydie & Suelle: Alchemists of the Myst...,Atelier Lydie & Suelle: Alchemists of the Myst...,0,2017,\N,5.9,11.0,['Atelier Lydie & Suelle: Alchemists of the My...,Atelier Lydie & Suelle: Alchemists of the Myst...,Atelier Lydie & Suelle: The Alchemists and the...,92.187500,1757,1465,[3025-647],Atelier Lydie & Suelle: The Alchemists and the...,0,[Atelier Lydie & Soeur: Fushigi na Kaiga no Re...,"['GUST Co., Ltd.']",['Role-Playing'],"['PC', 'PlayStation Vita', 'PlayStation Networ...",['Koei Tecmo'],NaN,Atelier Lydie & Suelle: The Alchemists and the...
4006860,tt15562384,videoGame,Mythic Heroes,Mythic Heroes,0,2021,Action,8.2,7.0,['Mythic Heroes'],Mythic Heroes,Mystic Heroes,92.307692,16855,13841,[3025-4663],Mystic Heroes,0,[nan],['Koei'],"['Action', 'Fighting']","['PlayStation 2', 'GameCube', 'PlayStation Net...","['Koei', 'Koei Tecmo Europe Ltd.']","['ESRB: T', 'CERO: All Ages']",Mystic Heroes
5544883,tt2159804,videoGame,Toypop,Toypop,0,1986,"Action,Family",7.2,10.0,"['Toypop', 'Toipoppu']",Toypop,Toy Pop,92.307692,28450,23237,[3025-3081],Toy Pop,1986,[nan],['Namco'],['Shooter'],"['PlayStation', 'Arcade', 'Wii Shop', 'PlaySta...","['Bandai Namco Entertainment', 'Namco', 'HAMST...",NaN,Toy Pop
3058720,tt13818966,videoGame,Bum Simulator,Bum Simulator,0,2019,"Action,Adventure,Comedy",6.8,12.0,['Bum Simulator'],Bum Simulator,BMX Simulator,92.307692,1917,1599,"[3025-4645, 3025-1047]",BMX Simulator,1988,[nan],['Codemasters'],"['Sports', 'Driving/Racing']","['Amiga', 'Amstrad CPC', 'Atari ST', 'Commodor...",['Codemasters'],NaN,BMX Simulator
1769121,tt11442976,videoGame,iRacing,iRacing,0,2008,Sport,7.9,11.0,['iRacing'],iRacing,Racing,92.307692,20498,16799,"[3025-1173, 3025-1171]",Racing,0,[Pro Racer\r\nSimple 1500 Series Vol. 13: The ...,['Tamsoft Corporation'],['Driving/Racing'],"['PlayStation', 'Play

At 92% ratio, 14 of the 20 games appear to be matching with the correct results (70%). By the time we up the ratio to 95% we are running with an estimated 90-95% matching accuracy (See below).

In [177]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 95].sort_values("best_fit_ratio").head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
2991931,tt1369557,videoGame,Race,Race,0,2006,Sport,7.0,5.0,"['Race: Official WTCC Game', 'Race']",Race: Official WTCC Game,Race 07: The Official WTCC Game,95.0,20491,16793,[3025-1757],Race 07: The Official WTCC Game,0,[nan],['SimBin Development Team AB'],"['Driving/Racing', 'Simulation']",['PC'],['SimBin Development Team AB'],['ESRB: E'],Race 07: The Official WTCC Game
1660294,tt11244996,videoGame,Dracula 4: The Shadow of the Dragon,Dracula 4: The Shadow of the Dragon,0,2013,"Adventure,Fantasy,Horror",6.2,6.0,['Dracula 4: The Shadow of the Dragon'],Dracula 4: The Shadow of the Dragon,Dracula 4: Shadow of the Dragon,95.0,6904,5674,[3025-1527],Dracula 4: Shadow of the Dragon,0,[nan],['Anuman Interactive'],['Adventure'],"['Mac', 'PC', 'iPhone', 'Android']",['Microïds'],NaN,Dracula 4: Shadow of the Dragon
1657809,tt11240358,videoGame,PewDiePie: Legend of the Brofist,PewDiePie: Legend of the Brofist,0,2015,"Action,Adventure,Fantasy",7.1,31.0,['PewDiePie: Legend of the Brofist'],PewDiePie: Legend of the Brofist,PewDiePie: Legend of Brofist,95.0,18894,15527,[3025-4778],PewDiePie: Legend of Brofist,0,[nan],['Outerminds Inc.'],['Platformer'],"['PC', 'iPhone', 'iPad', 'Android']",['Outerminds Inc.'],NaN,PewDiePie: Legend of Brofist
5157346,tt2002795,videoGame,Space Wars,Space Wars,0,1977,"Action,Sci-Fi",6.3,24.0,"['Space Wars', 'Space War']",Space Wars,3D Space Wars,95.0,230,205,[3025-4784],3D Space Wars,1983,[nan],['Graftgold Ltd.'],"['Action', 'Shooter']","['ZX Spectrum', 'Dragon 32/64']","['Hewson Consultants Ltd.', 'Riko Data']",NaN,3D Space Wars
4068233,tt15671470,videoGame,Mortal Kombat 11: Ultimate,Mortal Kombat 11: Ultimate,0,2020,"Action,Adventure,Fantasy",8.3,191.0,['Mortal Kombat 11: Ultimate'],Mortal Kombat 11: Ultimate,Ultimate Mortal Kombat,95.0,28965,23670,[3025-7],Ultimate Mortal Kombat,0,[UMK],"['Midway Games', 'Other Ocean Interactive']",['Fighting'],['Nintendo DS'],['Midway Games'],['ESRB: M'],Ultimate Mortal Kombat
7822311,tt5733684,videoGame,Deluxe Ski Jump 2,Deluxe Ski Jump 2,0,2001,Sport,6.8,16.0,['Deluxe Ski Jump 2'],Deluxe Ski Jump 2,Deluxe Ski Jump,95.0,5796,4767,[3025-3777],Deluxe Ski Jump,0,[nan],['Mediamond'],"['Simulation', 'Snowboarding/Skiing']",['PC'],['Mediamond'],NaN,Deluxe Ski Jump
5164418,tt2009554,videoGame,Pokémon Diamond Version,Poketto Monsutâ Daiyamondo,0,2006,"Action,Adventure,Family",8.1,940.0,"['Pokémon Diamant-Edition', 'Pokémon Edición D...",Pokémon Diamond,Pokémon Diamond/Pearl,95.0,19361,15908,[3025-452],Pokémon Diamond/Pearl,0,[Pokemon Diamond/Pearl],"['Game Freak, Inc.']",['Role-Playing'],['Nintendo DS'],['Nintendo'],"['BBFC: PG', 'ESRB: E', 'PEGI: 3+', 'CERO: A']",Pokémon Diamond/Pearl
9053475,tt8456562,videoGame,Tennis World Tour,Tennis World Tour,0,2018,\N,5.6,30.0,['Tennis World Tour'],Tennis World Tour,Virtua Tennis: World Tour,95.0,29455,24061,"[3025-2870, 3025-208]",Virtua Tennis: World Tour,0,[nan],['Sumo Digital Ltd.'],['Sports'],['PlayStation Portable'],['Sega'],['ESRB: E'],Virtua Tennis: World Tour
329523,tt0344216,videoGame,Pokémon Sapphire Version,Poketto Monsutâ Safaia,0,2002,"Action,Adventure,Family",8.3,921.0,"['Pokémon: Sapphire Version', 'Pokémon Sapphir...",Pokémon Sapphire,Pokémon Ruby/Sapphire,95.0,19429,15948,[3025-452],Pokémon Ruby/Sapphire,0,[Pokémon Ruby\nPokémon Sapphire\nPokemon Ruby/...,"['Creatures, Inc.', 'Game Freak, Inc.']",['Role-Playing'],['Game Boy Advance'],"['Nintendo', 'The Pokémon Company']","['CERO: A', 'PEGI: 3+', 'ESRB: E', 'OFLC: G8+']",Pokémon Ruby/Sapphire
331726,tt0346478,videoGame,The Adventures of Pinocchio,The Adventures of Pinocchio,0,1996,"Adventure,Family,Fantasy",6.6,85.0,['The Adventures of Pinocchio'],The Adventures of Pinocchio,Adventures of Pinocchio,95.0,635,541,[3025-

At 93%, I'd estimate 17/20 in the correct classification, which is 85% accuracy.

In [178]:
copy_of_imdb_games[copy_of_imdb_games["best_fit_ratio"] >= 93].sort_values("best_fit_ratio").head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,akas,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,aliases,developers,genres_y,platforms,publishers,rating,all_names_y
1057122,tt1016087,videoGame,Sam and Max: Abe Lincoln Must Die!,Sam and Max: Abe Lincoln Must Die!,0,2007,Adventure,8.4,118.0,"['Sam & Max: Abraham Lincoln Doit Mourir!', 'S...",Sam and Max Episode 104: Abe Lincoln Must Die!,Sam & Max Episode 4: Abe Lincoln Must Die!,93.023256,21913,17954,[3025-380],Sam & Max Episode 4: Abe Lincoln Must Die!,0,[nan],['Telltale Games'],['Adventure'],"['Wii', 'Xbox 360 Games Store', 'PC']",['Telltale Games'],NaN,Sam & Max Episode 4: Abe Lincoln Must Die!
292629,tt0305792,videoGame,Final Fantasy Legend 3,SaGa 3: Jikû no hasha,0,1993,"Adventure,Fantasy",7.1,44.0,"['SaGa 3: Jikû no hasha', 'Final Fantasy Legen...",Final Fantasy Legend 3,Final Fantasy Legend II,93.333333,8837,7202,"[3025-705, 3025-194]",Final Fantasy Legend II,0,[SaGa 2: Hihou Densetsu\nSaGa 2: Hihou Densets...,['Squaresoft'],['Role-Playing'],['Game Boy'],"['Squaresoft', 'Sunsoft']",['CERO: A'],Final Fantasy Legend II
2996095,tt1370352,videoGame,Badlands,Badlands,0,1989,"Action,Sci-Fi,Sport",6.0,12.0,['Badlands'],Badlands,Badland,93.333333,1995,1666,[3025-4052],Badland,0,[nan],['Frogmind Games'],['Platformer'],"['Mac', 'PlayStation Network (PS3)', 'PC', 'iP...",NaN,['ESRB: E'],Badland
3350481,tt14349502,videoGame,Doctor Who the Lonely Assassins,Doctor Who the Lonely Assassins,0,2021,"Drama,Sci-Fi",7.5,48.0,"['Doctor Who the Lonely Assassins', 'Doctor Wh...",Doctor Who: The Lonely Assassins,Dr Who: The Lonely Assassins,93.333333,6866,5642,[3025-1650],Dr Who: The Lonely Assassins,0,[nan],['Kaigan Games'],['Adventure'],"['PC', 'iPhone', 'Android', 'Nintendo Switch']","['Maze Theory', 'Another Indie']",NaN,Dr Who: The Lonely Assassins
5800398,tt2263622,videoGame,Platoon,Platoon,0,1987,"Action,War",6.1,42.0,['Platoon'],Platoon,Splatoon,93.333333,24028,19660,[3025-3755],Splatoon,0,[nan],"['Nintendo EAD', 'Monolith Software, Inc.']","['Action', 'Shooter', 'Platformer']",['Wii U'],"['Nintendo', 'Maxsoft']","['ESRB: E10+', 'PEGI: 7+', 'CERO: A', 'OFLC: PG']",Splatoon
5797644,tt2261285,videoGame,Badlands,Badlands,0,1984,"Action,Adventure,Animation",7.5,13.0,"['Badlands', 'Baddoranzu']",Badlands,Badland,93.333333,1995,1666,[3025-4052],Badland,0,[nan],['Frogmind Games'],['Platformer'],"['Mac', 'PlayStation Network (PS3)', 'PC', 'iP...",NaN,['ESRB: E'],Badland
8725634,tt7743938,videoGame,Paladins: Champions of the Realm,Paladins: Champions of the Realm,0,2016,Action,6.6,443.0,"['Paladins: Champions of the Realm', 'Paladins']",Paladins,Paladin,93.333333,18577,15281,[3025-3196],Paladin,1988,[nan],['Omnitrend Software'],"['Strategy', 'Role-Playing']","['Amiga', 'Atari ST', 'PC']","['Omnitrend Software', 'Cascade Games']",NaN,Paladin
5124516,tt1985990,videoGame,Dungeon,Dungeon,0,1975,Fantasy,8.1,12.0,['Dungeon'],Dungeon,Dungeons,93.333333,7414,6080,[3025-4007],Dungeons,0,[nan],['Realmforge Studios GmbH'],"['Strategy', 'Simulation']",['PC'],['Kalypso Media'],NaN,Dungeons
2323320,tt1244689,videoGame,F1 Race,F1 Race,0,1984,Sport,5.4,30.0,['F1 Race'],F1 Race,F-1 Race,93.333333,8260,6733,[3025-805],F-1 Race,0,[nan],"['Nintendo R&D1', 'HAL Laboratory, Inc.', 'SRD...",['Driving/Racing'],"['Game Boy', 'Nintendo Entertainment System']",['Nintendo'],NaN,F-1 Race
1643198,tt11213376,videoGame,Outlive,Outlive,0,2001,Sci-Fi,7.5,6.0,['Outlive'],Outlive,Out Live,93.333333,18276,15021,[3025-3987],Out Live,0,[nan],['Sunsoft'],['Role-Playing'],"['TurboGrafx-16', 'Wii Shop']",['Sunsoft'],['CERO: A'],Out Live


In [242]:
copy_of_imdb_games.tconst[(copy_of_imdb_games["best_fit_ratio"] >= 0) & (copy_of_imdb_games["best_fit_ratio"] <= 95)].count()

6260

In [192]:
copy_of_imdb_games.tconst[(copy_of_imdb_games["best_fit_ratio"] >= 95.1) & (copy_of_imdb_games["best_fit_ratio"] <= 100)].count()

8124

In [275]:
copy_of_imdb_games.count()

tconst              14384
titleType           14384
primaryTitle        14384
originalTitle       14384
isAdult             14384
startYear           14384
genres_x            14384
averageRating       14384
numVotes            14384
all_names_x         14384
best_fit_title      14384
best_fit_ratio      14384
best_fit_game_id    14384
game_id             14384
franchise_id        14384
name                14384
release_year        14384
developers          14090
genres_y            14162
platforms           14323
publishers          14173
rating               8685
dtype: int64

In [276]:
def remove_nonmatches(row):
    if row.best_fit_ratio < 85:
        row.best_fit_game_id = np.nan
        row.game_id = np.nan
        row.franchise_id = np.nan
        row.developers = np.nan
        row.genres_y = np.nan
        row.platforms = np.nan
        row.publishers = np.nan
        row.rating = np.nan
    return row

In [277]:
copy_of_imdb_games = copy_of_imdb_games.apply(remove_nonmatches, axis = 1)

In [ ]:
#UNUSED  TO DELETE ##
copy_of_imdb_games2[((copy_of_imdb_games2['release_year'] != copy_of_imdb_games2['startYear']) & (copy_of_imdb_games2['release_year'] != "0"))].sort_values("tconst")

### AT THIS STAGE:

All the games with a very similar match (currently set to 85%) have the relevant franchise_ids applied to their data. The next thing to do will be to extract the franchise_ids and do one of the following:
Either, each unique franchise_id has its own index, with multiple movies
Or, each unique movie will have multiple franchises
Or, Each unique movie will have one row, with the franchises stored as a list in those rows.

In [278]:
copy_of_imdb_games.sample()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres_x,averageRating,numVotes,all_names_x,best_fit_title,best_fit_ratio,best_fit_game_id,game_id,franchise_id,name,release_year,developers,genres_y,platforms,publishers,rating
9015994,tt8375994,videoGame,The Silver Case,The Silver Case,0,1999,"Crime,Horror,Mystery",6.8,22.0,The Silver Case,The Silver Case,100.0,23008.0,18842.0,[3025-2688],Silver Jiken,0,['Grasshopper Manufacture inc.'],['Adventure'],"['PlayStation', 'PlayStation Network (PS3)', '...","['ASCII Entertainment Software, Inc.']",['CERO: D']


Cleaning the genres: - Convert to list.

In [279]:
### UNUSED - TO REMOVE
def remove_nested_lists(entry):
    if pd.isna(entry):
        return entry
    else:
        return ast.literal_eval(entry)

copy_of_imdb_games["genres_y"] = copy_of_imdb_games["genres_y"].apply(lambda x: remove_nested_lists(x))

type(copy_of_imdb_games.genres_y.iloc[0])


list

# DONE TO HERE

In [ ]:
giantbomb_games

Having done this, we know that with any duplicates in our giantbomb, we'll have no way of distinguishing between them when it comes to combining them into our imdb dataset. If we try to combine by name, we'll be matching 2+ giantbomb games to a single imdb result. So we need to find a way to combine the giantbomb duplicates into a single entry. The other issue would be that we don't know from an imdb entry whether 

In [ ]:
imdb_games[imdb_games.duplicated(subset = "primaryTitle", keep=False)]

In [ ]:
#dups = giantbomb_games[giantbomb_games.duplicated(subset='name', keep=False)]
#gb_duplicate_names = giantbomb_games[giantbomb_games.duplicated(subset='name', keep=False)] # all duplicate names
helpme = giantbomb_games[~giantbomb_games.game_id.isin(imdb_games.game_id)]

In [ ]:
helpme[helpme.duplicated(subset = "name", keep=False)]

In [ ]:
gb_duplicate_names = giantbomb_games.duplicated(subset='name', keep=False)
release_year_int = giantbomb_games['release_year'].astype(int).eq(0)
giantbomb_games = giantbomb_games.loc[~(gb_duplicate_names & release_year_int)]

In [ ]:
giantbomb_games.count()

In [ ]:
type(gb_duplicates)

Remove any duplicates from our giantbomb database where we have no record of the year of release

In [ ]:
gb_duplicate_names = giantbomb_games.duplicated(subset='name', keep=False)
release_year_int = giantbomb_games['release_year'].astype(int).eq(0)
gb_duplicates = giantbomb_games.loc[gb_duplicate_names & release_year_int]
#Having got all the duplicates with no release year, we'll drop them from the dataframe
giantbomb_games = giantbomb_games[~giantbomb_games.index.isin(gb_duplicates.index)]

In [ ]:
imdb_games = imdb_games[~imdb_games['tconst'].isin(imdb_duplicates['tconst'])]
imdb_games.tconst.count()

This leaves me with 13866 games in the imdb dataset to play with. The next step is to combine the imdb dataset with the giantbomb

In [ ]:
giantbomb_games.name.count()

In [ ]:
giantbomb_games.name.nunique()

The way I am going to do this is to first combine the rows where the year and name matches. Then combine any remaining rows where the name matches but the year doesn't match.

In [ ]:
giantbomb_games[giantbomb_games["game_id"] == 48320]

In [ ]:
copy_of_imdb_games = imdb_games
copy_of_giantbomb_games = giantbomb_games
print("Total imdb:{}, Total giantbomb:{}".format(copy_of_imdb_games.tconst.count(), copy_of_giantbomb_games.name.count()))

First get all the games that have a matching title and year. Merge them and remove from the original databases so they can't be used again.

In [ ]:
mergedDF = pd.DataFrame()

In [ ]:
mergedDF = pd.merge(copy_of_imdb_games, copy_of_giantbomb_games, left_on=["primaryTitle", "startYear"], right_on=["name", "release_year"], how="inner")
print("Total imdb: {}, Total giantbomb: {}, Total in df: {}".format(copy_of_imdb_games.tconst.count(), copy_of_giantbomb_games.name.count(), mergedDF.tconst.count()))

Now that we've matched the title and years, we'll drop any duplicates which contain years from the giantbomb data.

In [ ]:
copy_of_giantbomb_games = copy_of_giantbomb_games.drop_duplicates("name", keep=False)

In [ ]:
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
copied_imdb_mask.value_counts()

Now we have no duplicate names, we can attempt to match on just name, first with primaryTitle, then originalTitle

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["primaryTitle"], right_on=["name"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)

In [ ]:
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
copied_imdb_mask.value_counts()

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["originalTitle"], right_on=["name"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)

In [ ]:
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
copied_imdb_mask.value_counts()

In [ ]:
copied_giantbomb_mask.value_counts()

Now, check every possible alias in the giantbomb dataframe for a direct match.

In [ ]:
#Explode the aliases
copy_of_giantbomb_games = copy_of_giantbomb_games.explode("aliases")

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["primaryTitle"], right_on=["aliases"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)

In [ ]:
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
copied_imdb_mask.value_counts()

In [ ]:
copied_giantbomb_mask.value_counts()

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["originalTitle"], right_on=["aliases"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)

In [ ]:
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
copied_imdb_mask.value_counts()

In [ ]:
copied_giantbomb_mask.value_counts()

In [ ]:

copy_of_imdb_games['primaryTitle'] = copy_of_imdb_games['primaryTitle'].str.lower()
copy_of_imdb_games['originalTitle'] = copy_of_imdb_games['originalTitle'].str.lower()

copy_of_giantbomb_games['name'] = copy_of_giantbomb_games['name'].str.lower()
copy_of_giantbomb_games['aliases'] = copy_of_giantbomb_games['aliases'].str.lower()

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["primaryTitle"], right_on=["name"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["originalTitle"], right_on=["name"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])
print(copied_imdb_mask.value_counts())

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["primaryTitle"], right_on=["aliases"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])
print(copied_imdb_mask.value_counts())

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["originalTitle"], right_on=["aliases"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])
print(copied_imdb_mask.value_counts())

In [ ]:
import re
temp = mergedDF[mergedDF["name"].str.contains("Sherlock", na=False, flags=re.IGNORECASE, regex=True)]
temp

In [ ]:
#Explode the akas

copy_of_imdb_games = copy_of_imdb_games.explode("akas")

In [ ]:
copy_of_imdb_games['akas'] = copy_of_imdb_games['akas'].str.lower()

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["akas"], right_on=["name"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])
print(copied_imdb_mask.value_counts())

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games[~copied_imdb_mask], copy_of_giantbomb_games[~copied_giantbomb_mask], left_on=["akas"], right_on=["aliases"], how="inner")
mergedDF = pd.concat([mergedDF, mergedDF2], axis=0)
copied_imdb_mask = copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])
copied_giantbomb_mask = copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])
print(copied_imdb_mask.value_counts())

Create a function that combines all the possible names including akas, into a list foe each imdb entry

In [ ]:
nonmatched_imdb = copy_of_imdb_games[~copied_imdb_mask]
nonmatched_gb = copy_of_giantbomb_games[~copied_giantbomb_mask]

In [ ]:
def imdb_all_names(imdb_row):
    imdb_row_list = [imdb_row.primaryTitle, imdb_row.originalTitle]
    if type(imdb_row.akas) ==str:
        akas_list = ast.literal_eval(imdb_row.akas)
        for x in akas_list:
            imdb_row_list.append(x)
    return imdb_row_list

In [ ]:
nonmatched_imdb['all_names'] = nonmatched_imdb.apply(lambda row: imdb_all_names(row), axis=1)
nonmatched_imdb

In [ ]:
nonmatched_gb['all_names'] = nonmatched_gb.apply(lambda row: gb_all_names(row), axis=1)
nonmatched_gb

In [ ]:
def gb_all_names(gb_row):
    gb_row_list = [gb_row["name"]]
    for alias in gb_row.aliases:
        if type(alias) == str:
            gb_row_list.append(alias) 
    return gb_row_list

In [ ]:
nonmatched_imdb['all_names'] = nonmatched_imdb['all_names'].apply(lambda x: list(set(x)))
nonmatched_gb['all_names'] = nonmatched_gb['all_names'].apply(lambda x: list(set(x)))

In [ ]:
nonmatched_imdb = nonmatched_imdb.explode("all_names")
nonmatched_gb = nonmatched_gb.explode("all_names")

In [ ]:

nonmatched_gb = nonmatched_gb.explode("all_names")
nonmatch_gb_list = nonmatched_gb.all_names.to_list()

In [ ]:
def get_top_unmatches(imdb_row):
    return process.extract(imdb_row.all_names, nonmatch_gb_list, limit = 3)

In [ ]:
time_at_start = time.perf_counter()
nonmatched_imdb['best_matches'] = nonmatched_imdb.apply(lambda row: get_top_unmatches(row), axis = 1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
nonmatched_imdb

In [ ]:
time_at_start = time.perf_counter()
nonmatched_imdb['best_fit'] = nonmatched_imdb.apply(lambda row: best_fit(row), axis = 1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
time_at_start = time.perf_counter()
nonmatched_imdb[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = nonmatched_imdb['best_fit'].apply(lambda x: pd.Series([i for i in x]))
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
nonmatched_imdb = nonmatched_imdb.sort_values(["best_fit_ratio"])
nonmatched_imdb

In [ ]:
nonmatched_imdb["best_fit_ratio"].value_counts()

In [ ]:
nonmatched_imdb = nonmatched_imdb[nonmatched_imdb['best_fit_ratio'] >= 80]
nonmatched_imdb

In [ ]:
copy_of_imdb_games['all_names'] = copy_of_imdb_games.apply(lambda row: imdb_all_names(row), axis=1)
copy_of_imdb_games

Combine all giantbomb possible names including aliases.

In [ ]:
def gb_all_names(gb_row):
    gb_row_list = [gb_row["name"]]
    for alias in gb_row.aliases:
        if type(alias) == str:
            gb_row_list.append(alias) 
    return gb_row_list


In [ ]:
copy_of_giantbomb_games['all_names'] = copy_of_giantbomb_games.apply(lambda row: gb_all_names(row), axis=1)
copy_of_giantbomb_games

Remove any duplicates in all the name lists

In [ ]:
copy_of_imdb_games['all_names'] = copy_of_imdb_games['all_names'].apply(lambda x: list(set(x)))
copy_of_giantbomb_games['all_names'] = copy_of_giantbomb_games['all_names'].apply(lambda x: list(set(x)))

In [ ]:
copy_of_giantbomb_games.sample()

In [ ]:
copy_of_imdb_games.drop(["best_matches", "best_fit", "best_fit_title", "best_fit_ratio", "best_fit_game_id"])

## Attempt to use rapidFuzz

This seems to be the fastest way we can fuzzymatch across the entire dataframe (that I have found). We first explode all the giantbomb names, and the imdb names. Then convert the giantbomb names to a list. We can then run rapidfuzz for all the names in our imdb dataset against the giantbomb list and append the results to a best_matches column in our imdb dataframe. Once we have this we can 

In [ ]:
copy_of_imdb_games = copy_of_imdb_games[~copied_imdb_mask].explode("all_names")
copy_of_giantbomb_games = copy_of_giantbomb_games[~copied_giantbomb_mask].explode("all_names")

In [ ]:
giantbomb_remaining_list = copy_of_giantbomb_games.all_names.to_list()

In [ ]:
from rapidfuzz import process, fuzz

In [ ]:
def get_top_matches(imdb_row):
    return process.extract(imdb_row.all_names, giantbomb_remaining_list, limit = 3)

In [ ]:
import time

In [ ]:
time_at_start = time.perf_counter()
copy_of_imdb_games['best_matches'] = copy_of_imdb_games.apply(lambda row: get_top_matches(row), axis = 1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt0383279"]

In [ ]:
print(copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt0383279"]["best_matches"].iloc[0])
print(copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt0383279"]["best_matches"].iloc[1])
print(copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt0383279"]["best_matches"].iloc[2])

In [ ]:
type(copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt0383279"]["best_matches"].iloc[0][0][0])

In [ ]:
def best_fit(row):
    highest = 0
    best_match = ()
    for match in row["best_matches"]:
        if match[1] > highest:
            highest = match[1]
            best_match = match
    return best_match

In [ ]:
time_at_start = time.perf_counter()
copy_of_imdb_games['best_fit'] = copy_of_imdb_games.apply(lambda row: best_fit(row), axis = 1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
copy_of_imdb_games[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = copy_of_imdb_games['best_fit'].apply(lambda x: pd.Series([i for i in x]))


In [ ]:
copy_of_imdb_games = copy_of_imdb_games[copy_of_imdb_games['best_fit_ratio'] >= 90.5]

In [ ]:
copy_of_imdb_games =copy_of_imdb_games.sort_values(["tconst", "best_fit_ratio"])
copy_of_imdb_games.head(50)

In [ ]:
small_imdb_exploded['best_fit'] = small_imdb_exploded.apply(lambda row: best_fit(row), axis = 1)


In [ ]:
small_imdb_exploded[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = small_imdb_exploded['best_fit'].apply(lambda x: pd.Series([i for i in x]))

    

In [ ]:
frame = copy_of_imdb_games[copy_of_imdb_games["tconst"]=="tt7025920"]
frame

In [ ]:
copy_of_giantbomb_games.game_id.count()

In [ ]:
copy_of_imdb_games.to_csv("copy_of_imdb_games.csv")

In [ ]:
copy_of_imdb_games = pd.read_csv("copy_of_imdb_games.csv")

In [ ]:
copy_of_imdb_games['best_matches'] = copy_of_imdb_games["best_matches"].apply(lambda row: ast.literal_eval(row))
type(copy_of_imdb_games["best_matches"].iloc[0])

In [ ]:
copy_of_imdb_games

In [ ]:
copy_of_imdb_games['best_fit'] = copy_of_imdb_games.apply(lambda row: best_fit(row), axis = 1)
copy_of_imdb_games[['best_fit_title', 'best_fit_ratio', 'best_fit_game_id']] = copy_of_imdb_games['best_fit'].apply(lambda x: pd.Series([i for i in x]))
copy_of_imdb_games = copy_of_imdb_games.sort_values("best_fit_ratio")
copy_of_imdb_games

In [ ]:
copy_of_imdb_games = copy_of_imdb_games.sort_values("best_fit_ratio")

In [ ]:
copy_of_imdb_games[copy_of_imdb_games["tconst"] == "tt11696274"].iloc[1]["best_matches"]

In [ ]:
copy_of_giantbomb_games[copy_of_giantbomb_games.index == 18179]

In [ ]:
copy_of_imdb_games[copy_of_imdb_games["tconst"] == "tt11696274"]

In [ ]:
copy_of_giantbomb_games

In [ ]:
mergedDF[mergedDF["game_id"] == 48320]

In [ ]:
copy_of_giantbomb_games[copy_of_giantbomb_games["game_id"] == "48320"]

In [ ]:
copy_of_imdb_games[(copy_of_imdb_games["best_fit_ratio"] >97) &  (copy_of_imdb_games["best_fit_ratio"] <98)].head(50)

In [ ]:
import numpy as np

In [ ]:
small_imdb_exploded['best_matches'] = small_imdb_exploded.apply(lambda row: get_top_matches(row), axis = 1)


In [ ]:
small_imdb_exploded

In [ ]:
small_imdb_exploded["best_matches"].iloc[0]

In [ ]:
def withlist(imdb_row):
    results_list = process.extract(imdb_row.all_names, giantbomb_list, limit = 3)
    return results_list

In [ ]:
import time
time_at_start = time.perf_counter()
small_imdb_exploded['best_matches'] = small_imdb_exploded.apply(lambda row: withlist(row), axis=1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
small_imdb_exploded

In [ ]:
giantbomb_exploded.iloc[22642]

In [ ]:
small_imdb_exploded.best_matches.iloc[0]

In [ ]:
def rapidfuzzymatch(imdb_row):
    results_list = []
    for index, row in giantbomb_games.iterrows():
        distances  = process.cdist(row["all_names"], imdb_row.all_names, score_cutoff = 87)
        top_indices = np.argsort(distances, axis=1)
        top_values = imdb_row[top_indices]
        print(top_values)
            
            #for result in results:
                #Get tuple value if above 95
            #    if result[1]>87:
            #        results_list.append(set([row["game_id"], row["name"], result[1]]))
    return results_list

In [ ]:
import time
time_at_start = time.perf_counter()
small_imdb['best_matches'] = small_imdb.apply(lambda row: rapidfuzzymatch(row), axis=1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
small_imdb.best_matches.iloc[0]

## Attempting to use fuzzy matching

The next step is to attempt to fuzzy match the titles and the original titles from the imdb dataset against the giantbomb database

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Fuzzy match attempt

In [ ]:
def fuzzymatch(imdb_row):
    results_list = []
    for index, row in giantbomb_games.iterrows():
        for item in row["all_names"]:
            results = process.extract(item, imdb_row.all_names, limit = 3)
            for result in results:
                #Get tuple value if above 95
                if result[1]>87:
                    results_list.append(set([row["game_id"], row["name"], result[1]]))
    return results_list


Testing on single entry

In [ ]:
small_imdb = imdb_games[imdb_games["tconst"]== "tt7990520"]
small_imdb

In [ ]:
import time
time_at_start = time.perf_counter()
small_imdb['best_matches'] = small_imdb.apply(lambda row: fuzzymatch(row), axis=1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

In [ ]:
small_imdb.best_matches.iloc[0]

In [ ]:
small_imdb

### Next attempt, explode the dataframes, then do fuzzymatch

In [ ]:
small_imdb_exploded = small_imdb.explode("all_names")

In [ ]:
small_imdb_exploded

In [ ]:
def fuzzymatch_exploded(imdb_row):
    results_list = []
    for index, row in giantbomb_exploded.iterrows():
        results = process.extract(row["all_names"], imdb_row.all_names, limit = 3)
        for result in results:
            #Get tuple value if above 95
            if result[1]>87:
                results_list.append(set([row["game_id"], row["name"], result[1]]))
    return results_list

In [ ]:
imdb_exploded = imdb_games.explode("all_names")
imdb_exploded.count()

In [ ]:
giantbomb_exploded = giantbomb_games.explode("all_names")
giantbomb_exploded.count()

In [ ]:
import time
time_at_start = time.perf_counter()
small_imdb['exploded_best_matches'] = small_imdb.apply(lambda row: fuzzymatch_exploded(row), axis=1)
time_at_end = time.perf_counter()
print(time_at_end - time_at_start)

## Attempting to use difflib

In [ ]:
string =small_imdb_exploded.all_names.iloc[0]
close_matches = difflib.get_close_matches(string, possibilities=giantbomb_exploded['all_names'].tolist(), n=10)


In [ ]:
close_matches

In [ ]:
# https://stackoverflow.com/questions/56521625/quicker-way-to-perform-fuzzy-string-match-in-pandas

import difflib
from functools import partial

f = partial(difflib.get_close_matches, possibilities=giantbomb_exploded['all_names'].tolist(), n=1)

matches = small_imdb_exploded['all_names'].map(f).str[0].fillna('')
scores = [difflib.SequenceMatcher(None, x, y).ratio() 
    for x, y in zip(matches, small_imdb['all_names'])
]

small_imdb.assign(best=matches, score=scores)

# Consider exploding the all_names, then combining back but keeping the one with the best ratio?

## First attempt to merge, purely using dataframes

In [ ]:
mergedDF = pd.merge(copy_of_imdb_games, copy_of_giantbomb_games, left_on=["primaryTitle", "startYear"], right_on=["name", "release_year"], how="inner")
#copy_of_imdb_games = copy_of_imdb_games[~copy_of_imdb_games['tconst'].isin(mergedDF['tconst'])]
#copy_of_giantbomb_games = copy_of_giantbomb_games[~copy_of_giantbomb_games['game_id'].isin(mergedDF['game_id'])]
print("Total imdb: {}, Total giantbomb: {}, Total in df: {}".format(copy_of_imdb_games.tconst.count(), copy_of_giantbomb_games.name.count(), mergedDF.tconst.count()))

In [ ]:
Now match any where the names directly match, and the release_year is 0

In [ ]:
mergedDF2 = pd.merge(copy_of_imdb_games, copy_of_giantbomb_games[copy_of_giantbomb_games['release_year'] == "0"], left_on="primaryTitle", right_on="name", how="inner")
copy_of_imdb_games = copy_of_imdb_games[~copy_of_imdb_games['tconst'].isin(mergedDF2['tconst'])]
copy_of_giantbomb_games = copy_of_giantbomb_games[~copy_of_giantbomb_games['game_id'].isin(mergedDF2['game_id'])]
print("Total imdb: {}, Total giantbomb: {}, Total in df2: {}".format(copy_of_imdb_games.tconst.count(), copy_of_giantbomb_games.name.count(), mergedDF2.tconst.count()))

In [ ]:
mergedDF2[mergedDF2["primaryTitle"]=="Alice in Wonderland"]

In [ ]:
mergedDF3 = pd.merge(copy_of_imdb_games, copy_of_giantbomb_games, left_on="originalTitle", right_on="name", how="inner")
copy_of_imdb_games = copy_of_imdb_games[~copy_of_imdb_games['tconst'].isin(mergedDF3['tconst'])]
copy_of_giantbomb_games = copy_of_giantbomb_games[~copy_of_giantbomb_games['game_id'].isin(mergedDF3['game_id'])]
print("Total imdb: {}, Total giantbomb: {}, Total in df3: {}".format(copy_of_imdb_games.tconst.count(), copy_of_giantbomb_games.name.count(), mergedDF3.tconst.count()))

In [ ]:
mergedDF4 = pd.concat([mergedDF, mergedDF2, mergedDF3], axis=0)
mergedDF4

In [ ]:
temp = imdb_duplicates[~(imdb_duplicates["primaryTitle"]== imdb_duplicates["originalTitle"])]

In [ ]:
temp = pd.merge(imdb_duplicates, giantbomb_games, left_on=['primaryTitle'], right_on=['name'], how='inner')
temp

In [ ]:
giantbomb_games.sort_values("game_id").head()

In [ ]:
tempdb = pd.merge(imdb_duplicates, giantbomb_games, left_on=['primaryTitle'], right_on=['name'], how='inner')
tempdb

In [ ]:
for index1 in imdb_duplicates.primaryTitle:
    #rint(index1)
    value = fuzz.token_sort_ratio("bction 2", index1)
    if value > 80:
        print(index1)
        print(value)

In [ ]:
imdb_duplicates.head(500)

In [ ]:
for index1 in imdb_duplicates.primaryTitle:
    #rint(index1)
    value = fuzz.token_sort_ratio("bction 2", index1)
    if value > 80:
        print(index1)
        print(value)

In [ ]:
imdb_duplicates

In [ ]:
imdb_duplicates = imdb_duplicates[imdb_duplicates['primaryTitle'].isin(giantbomb_games['name'])]
imdb_duplicates

Create dictionary with count for each duplicate

In [ ]:
imdb_duplicate_counts

In [ ]:
imdb_duplicate_counts = imdb_duplicates.groupby('primaryTitle').size()
imdb_duplicate_dict = {k:v for k,v in imdb_duplicate_counts.items() if v > 1}
imdb_duplicate_dict

In [ ]:
giantbomb_games[giantbomb_games["name"] == "Worms"]


Note here, we are only going to consider direct matches of a name for brevity's sake. If giantbomb contains no records that match the name of an imdb duplicate, we will assume the duplicates are not part of a franchise.

In [ ]:
for key, value in imdb_duplicate_dict.items():
    if giantbomb_games.name[giantbomb_games["name"] == key].count() == 0:
        # Game is not a franchise game, and can be kept in original imdb dataframe
        print(key)
        #imdb_duplicates = imdb_duplicates.drop(index=imdb_duplicates.index[imdb_duplicates['primaryTitle'] == key])
imdb_duplicates.count()

In [ ]:
imdb_duplicates.count()

In [ ]:
Action 52 is not in giantbomb games so we can keep in our original database as it's a unique game. Therefore we need to drop it from our dups dataframe

This leaves us with 411 duplicates, which exist in the imdb database but we may not be able to accurately match them with the 

WE ARE CREATING A DUPS DATAFRAME WHICH WILL BE ALL THE DUPLICATES WE CAN'T DISTINGUISH BETWENN. IF WE CAN DISTINGUISH, WE DROP FROM THE DUPS DATABASE. IF IMDB GAMES NAME DOES NOT APPEAR IN GIANTBOMB GAMES NAME, DROP FROM DUPLICATES LIST

sO FAR WE HAVE DONE ENOUGH. wE JUST NEED TO DO FUZZY MATCHING ON THE NAMES TO COMPARE.EG. ADAM'S VENTURE EXISTS IN BOTH BUT NEEDS FUZZY MATCHING AGAINST ORIGINAL TITLE

In [ ]:
imdb_duplicates = imdb_duplicates[~imdb_duplicates['tconst'].isin(matching_imdb_dups['tconst'])]


In [ ]:
for key, value in imdb_duplicate_dict.items():
    if giantbomb_games.name[giantbomb_games["name"] == key].count() == 0:
        # Game is not a franchise game, and can be kept in original imdb dataframe
        imdb_duplicates = imdb_duplicates.drop(index=imdb_duplicates.index[imdb_duplicates['primaryTitle'] == key])
    elif giantbomb_games.name[giantbomb_games["name"] == key].count() != value:
        #matching_ids holds index values of duplicates
        matching_imdb_ids=imdb_duplicates.index[imdb_duplicates['primaryTitle'] == key]
        matching_gb_ids = giantbomb_games.index[giantbomb_games['name'] == key]
        if 
        #for id in matching_imdb_ids:
            #This extracts the year
         #   year =imdb_duplicates.startYear[imdb_duplicates.index[imdb_duplicates.index == id]].item()
          #  if giantbomb_games.name[giantbomb_games["name"] == key] and giantbomb_games.release_year[giantbomb_games["release_year"] == year]:
           #     for 
            #    print("Great success!", year)
           # else:
            #    print("poo")
        
        print(key, value, giantbomb_games.name[giantbomb_games["name"] == key].count())

In [ ]:
giantbomb_games[giantbomb_games.duplicated(["name"], keep=False)].count()

In [ ]:
import re
temp = imdb_games[imdb_games["primaryTitle"].str.contains("Batman", na=False, flags=re.IGNORECASE, regex=True)]
temp

In [ ]:
import re
temp2 = giantbomb_games[giantbomb_games["name"].str.contains("Adam's", na=False, flags=re.IGNORECASE, regex=True)]
temp2

In [ ]:
duplicate_giantbomb_names = giantbomb_games[giantbomb_games.duplicated(["name", "release_year"], keep=False)]
duplicate_giantbomb_names = duplicate_giantbomb_names.sort_values("name")
duplicate_giantbomb_names

Get list of all game_ids

In [ ]:
game_id_list = duplicate_giantbomb_names.index.to_list()

for each item in the list, get the corresponding json file. Clean to extract necessary data (esp. year)

In [ ]:
for game_id, api_url in game_api_dict.items():
    filename = "{}.json".format(game_id)
    if not Path(game_dir, filename).is_file():

        

In [ ]:
print(len(release_years))

In [ ]:
num_nans = id_year_df.isna().sum().sum()
print(num_nans)

In [ ]:
# group the data by name and count the number of occurrences
grouped = giantbomb_games.groupby(['name']).size().reset_index(name='count')

# select the rows where count > 1, indicating a duplicate name
duplicates = grouped[grouped['count'] > 1]

# select the rows with duplicate name and different game_id
duplicates = giantbomb_games[giantbomb_games['name'].isin(duplicates['name'])].groupby('name').filter(lambda x: x['game_id'].nunique() > 1)

# print the duplicate rows, sorted by name
if len(duplicates) > 0:
    print("Found {} rows with same name but different game_id:".format(len(duplicates)))
    print(duplicates.sort_values(by=['name', 'game_id']))
else:
    print("No rows with same name but different game_id found.")


In [ ]:
temp2 = temp[temp.duplicated(["name", "index"], keep=False)]
temp2 = temp2.sort_values("name")
temp2

In [ ]:
a = giantbomb_games[giantbomb_games["name"].str.contains('Sqrxz', na=False, flags=re.IGNORECASE, regex=True)]
a

The first thing I am going to do is check for any titles that appear only once in both dataframes and that have matching titles. With this criteria, we can assume that the games are the same. 

In [ ]:
unmatching_rows = imdb_games[~imdb_games['primaryTitle'].isin(giantbomb_games['name'])]

In [ ]:
try_this = unmatching_rows[unmatching_rows['originalTitle'].isin(giantbomb_games['name'])]
try_this.count()

In [ ]:
import re
temp = giantbomb_games[giantbomb_games["name"].str.contains('Infernal', na=False, flags=re.IGNORECASE, regex=True)]
temp.head(50)

My current plan for the most accurate way to approach this:

First check for duplicate names in both datasets. If there is a duplicate name in either dataset, we need to check whether this is because there are 2 games with the same name. If this is the case, we need a way to distinguish between the 2. I would anticipate year of release could work.

The next step would be comparing names against alternate titles. After this, any that still have no matches, would be subject to fuzzy matching.I'd anticipate at least 75% of the games in the imdb dataset should be contained in giantbomb's database.

In [ ]:
imdb_dups = imdb_games[imdb_games.duplicated(['primaryTitle', 'startYear'], keep=False)]
imdb_dups = imdb_dups.sort_values("primaryTitle")
print(imdb_dups)